# <center>Постановка задачи:<center>
* Исследовать данные и сделать выводы по полученным результатам
В ходе анализа нам необходимо:
* 1. Выяснить, какие факторы влияют на заработную плату специалиста DataScientist.
* 2. Ответить на ключевые вопросы:
   * Наблюдается ли ежегодный рост зарплат у специалистов Data Scientist?
   * Как соотносятся зарплаты Data Scientist и Data Engineer в 2022 году?
   * Как соотносятся зарплаты специалистов Data Scientist в компаниях различных размеров?
   * Есть ли связь между наличием должностей Data Scientist и Data Engineer и размером компании?


## <center>Загрузка и обработка данных<center>

In [1]:
# Импортируем основные библиотеки
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm 
from statsmodels import stats as sms
import matplotlib.pyplot as plt 
import seaborn as sns

In [2]:
# Загружаем данные
data = pd.read_csv('C:\IDE\data\ds_salaries.csv')
display(data.head())

,Unnamed: 0,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L


In [3]:
# Поиск и удаление дубликатов
print('Количество дубликатов: {}'.format(data[data.duplicated()].shape[0]))
data = data.drop_duplicates()
print('Размер таблицы после удаление дубликатов: {}'.format(data.shape))

Количество дубликатов: 0
Размер таблицы после удаление дубликатов: (607, 12)


**Вывод:**
* Дубликатов в данных не обнаруженно.

In [4]:
# Выведем общую информацию о датасете
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 607 entries, 0 to 606
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          607 non-null    int64 
 1   work_year           607 non-null    int64 
 2   experience_level    607 non-null    object
 3   employment_type     607 non-null    object
 4   job_title           607 non-null    object
 5   salary              607 non-null    int64 
 6   salary_currency     607 non-null    object
 7   salary_in_usd       607 non-null    int64 
 8   employee_residence  607 non-null    object
 9   remote_ratio        607 non-null    int64 
 10  company_location    607 non-null    object
 11  company_size        607 non-null    object
dtypes: int64(5), object(7)
memory usage: 57.0+ KB


**Вывод:**
   * 1. Типы данных признаков корректны
   * 2. Определенны числовые признаки (work_year, salary, salary_in_usd, remote_ratio)
   * 3. Определенны категориальные признаки (experience_level, employment_type, company_size, job_title, salary_currency,employee_residence, company_location)

In [5]:
# Удаляем неиформативные признаки
data = data.drop(['Unnamed: 0', 'salary', 'salary_currency'], axis=1)
display(data.head())

,work_year,experience_level,employment_type,job_title,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2020,MI,FT,Data Scientist,79833,DE,0,DE,L
1,2020,SE,FT,Machine Learning Scientist,260000,JP,0,JP,S
2,2020,SE,FT,Big Data Engineer,109024,GB,50,GB,M
3,2020,MI,FT,Product Data Analyst,20000,HN,0,HN,S
4,2020,SE,FT,Machine Learning Engineer,150000,US,50,US,L


**Выводы:**
   * 1. Признак Unnamed: 0 удален. Явно лишний признак, так как нумерация строк присутствует автоматически.
   * 2. Признак salary удален. Так как есть признак показывающий зарплату в долларах.
   * 3. Принзнак salary_currency удален. Раз есть признак показывающий зарплату в долларах, признак валюты не имеет ценности.

### Выводим статистические характеристики

In [7]:
# Для признака work_year
print(data['work_year'].mode())

0    2022
Name: work_year, dtype: int64


In [8]:
# Для признака remote_ratio
print(data['remote_ratio'].mode())

0    100
Name: remote_ratio, dtype: int64


In [16]:
# Для признака salary_in_usd
print(round(data['salary_in_usd'].describe()))

count       607.0
mean     112298.0
std       70957.0
min        2859.0
25%       62726.0
50%      101570.0
75%      150000.0
max      600000.0
Name: salary_in_usd, dtype: float64


In [10]:
# Для признака experience_level
print(data['experience_level'].describe())

count     607
unique      4
top        SE
freq      280
Name: experience_level, dtype: object


In [11]:
# Для признака employment_type
print(data['employment_type'].describe())

count     607
unique      4
top        FT
freq      588
Name: employment_type, dtype: object


In [12]:
# Для признака job_title
print(data['job_title'].describe())

count                607
unique                50
top       Data Scientist
freq                 143
Name: job_title, dtype: object


In [13]:
# Для признака employee_residence
print(data['employee_residence'].describe())

count     607
unique     57
top        US
freq      332
Name: employee_residence, dtype: object


In [14]:
# Для признака company_location
print(data['company_location'].describe())

count     607
unique     50
top        US
freq      355
Name: company_location, dtype: object


In [15]:
# Для для признака company_size
print(data['company_size'].describe())

count     607
unique      3
top         M
freq      326
Name: company_size, dtype: object


**Выводы:**
   * 1. Самый часто встречающийся год, в котором была выплачена зарплата - 2022.
   * 2. Наибольшее количество работы было выполнено удаленно. Самый частый показатель признака remote_ratio - 100.
   * 3. Средняя заработная плата равна 112298 долларам. Минимальная заработная плата равна 2859 долларам. Максимальный показатель заработной платы равен 600000 долларов.
   * 4. Самый распространенный показатель опыта работы это SE - Senior-level/Expert, встречается 280 раз.
   * 5. Самый расспространенный тип трудоустройства FT - полный рабочий день, втречается 588 раз.
   * 6. Самая распространенная должность Data Scientist, встречается 143 раза.
   * 7. Самая распространенная страна, в которой соискатель проживал в течении года, это US. Встречается 332 раза.
   * 8. Наибольшее число главных офисов работодателей находятся в US. Встречается 355 раз.
   * 9. Наибольшее количетсво компаний, имееют размер М - средняя компания. Показатель встречается 326 раз.

## <center>Визуальный анализ данных<center>

In [ ]:
fig, axes = sns.histplot(1, 1, figsize = (10, 5))
